In [81]:
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [82]:
# Load all songs
songs = []
for f in os.listdir("data/preprocessed"):
    songs.append(np.genfromtxt(("data/preprocessed/%s" % f), dtype=int, delimiter=','))

In [83]:
# Split data up into "patterns"
# normalize ints by dividing by 128
pattern_length = 100
data_X = []
data_y = []
for f in songs[:10]:
    for i in range(0, len(f) - pattern_length, 1):
        data_X.append(f[i:i+pattern_length])
        data_y.append(f[i+pattern_length])
n_patterns = len(data_X)
print("Total Patterns: ", n_patterns)

Total Patterns:  29293


In [84]:
# Find unused notes
freq = np.bincount(np.array(data_X).flatten())

# Get indices of nonzero frequencies
non_zero_freq = np.nonzero(freq)[0]

# Remember highest and lowest used notes
lowest = non_zero_freq[0]
highest = non_zero_freq[len(non_zero_freq) - 1]
n_notes = highest - lowest + 1

In [85]:
# Reshape X
X = np.reshape(data_X, (n_patterns, pattern_length, 1))

# Normalize
X = (X - lowest) / n_notes

# One hot encode
y = np_utils.to_categorical(data_y - lowest)

In [86]:
# LSTM
model = Sequential()
model.add(LSTM(512, input_shape=((pattern_length, 1)), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(n_notes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [87]:
# Checkpoints
checkpoint = ModelCheckpoint("checkpoint-{epoch:02d}.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)

In [88]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
29293/29293 [==============================] - 2845s 97ms/step - loss: 2.9267 - categorical_accuracy: 0.3790

Epoch 00001: loss improved from inf to 2.92672, saving model to checkpoint-01.hdf5
Epoch 2/20
29293/29293 [==============================] - 2806s 96ms/step - loss: 2.8387 - categorical_accuracy: 0.3805

Epoch 00002: loss improved from 2.92672 to 2.83868, saving model to checkpoint-02.hdf5
Epoch 3/20
29293/29293 [==============================] - 2716s 93ms/step - loss: 2.7346 - categorical_accuracy: 0.3805

Epoch 00003: loss improved from 2.83868 to 2.73456, saving model to checkpoint-03.hdf5
Epoch 4/20
29293/29293 [==============================] - 2724s 93ms/step - loss: 2.5094 - categorical_accuracy: 0.3854

Epoch 00004: loss improved from 2.73456 to 2.50943, saving model to checkpoint-04.hdf5
Epoch 5/20
29293/29293 [==============================] - 2725s 93ms/step - loss: 2.3118 - categorical_accuracy: 0.4005

Epoch 00005: loss improved from 2.50943 to 2.31181,

KeyboardInterrupt: 

In [89]:
# Load 
filename = "checkpoint-13.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [90]:
start = np.random.randint(0, len(data_X) - 1)
pattern=X[start]
song = []
#Generate
for i in range(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0).flatten()
    index=np.argmax(prediction)
    pattern = np.append(pattern, (index/float(n_notes)))
    pattern = pattern[1:len(pattern)]
    song = np.append(song, (index))

In [91]:
print((pattern*n_notes) + lowest)
np.savetxt("song.csv", (song + lowest), fmt='%s', delimiter=",")

[128.  86.  83.  82. 128.  82.  91.  83.  86.  91. 128. 128.  91.  86.
  83.  83.  86.  91. 128.  86.  83.  82. 128.  82.  91.  83.  86.  91.
 128. 128.  91.  86.  83.  83.  86.  91. 128.  86.  83.  82. 128.  82.
  91.  83.  86.  91. 128. 128.  91.  86.  83.  83.  86.  91. 128.  86.
  83.  82. 128.  82.  91.  83.  86.  91. 128. 128.  91.  86.  83.  83.
  86.  91. 128.  86.  83.  82. 128.  82.  91.  83.  86.  91. 128. 128.
  91.  86.  83.  83.  86.  91. 128.  86.  83.  82. 128.  82.  91.  83.
  86.  91.]
